# QUITS: Getting Started

This notebook is a README-style overview for the latest development version of QUITS. It shows how QUITS composes QLDPC codes, circuit-construction strategies, decoders, and noise models in one workflow.

## What Is Modular in QUITS?

- **Codes** (`quits.qldpc_code`): construct QLDPC families such as HGP, QLP, BPC, LCS, and BB.
- **Circuits** (`quits.qldpc_code.circuit_construction`): choose construction strategy per code.
- **Decoders** (`quits.decoder`): run sliding-window circuit-level or phenomenological decoding.
- **Noise models** (`quits.noise.ErrorModel`): configure idle, single-qubit, two-qubit, and SPAM error channels.

In [1]:
import numpy as np

from quits import CircuitBuildOptions
from quits.noise import ErrorModel
from quits.qldpc_code import BbCode, BpcCode, HgpCode
from quits.decoder import sliding_window_bposd_circuit_mem
from quits.simulation import get_stim_mem_result


## Circuit Strategy Matrix

| Code family | `zxcoloration` | `cardinal` | `custom` |
| --- | --- | --- | --- |
| HGP | yes | yes | no |
| QLP | yes | yes | no |
| BPC | yes | yes | no |
| LCS | yes | yes | no |
| BB | yes | no | yes |

- `zxcoloration` is available for all QLDPC codes.
- `cardinal` is available for HGP, QLP, BPC, and LCS.
- `custom` is available for BB code construction.

In [8]:
# Small HGP example with zxcoloration

d = 3
H = np.zeros((d, d), dtype=int)
for i in range(d):
    H[i, i] = 1
    H[i, (i + 1) % d] = 1

hgp = HgpCode(H, H)
zx_circuit = hgp.build_circuit(strategy="zxcoloration", num_rounds=1, basis="Z")
print("HGP zxcoloration depth:", hgp.depth)
print("Circuit instructions:", len(zx_circuit))


HGP zxcoloration depth: 8
Circuit instructions: 50


In [3]:
# Small BPC example with cardinal

bpc = BpcCode([0, 1, 2], [0, 4, 5], lift_size=6, factor=3)
cardinal_circuit = bpc.build_circuit(strategy="cardinal", num_rounds=1, basis="Z", seed=1)
print("BPC cardinal depth:", bpc.depth)
print("Circuit instructions:", len(cardinal_circuit))


BPC cardinal depth: 8
Circuit instructions: 70


In [4]:
# Small BB example with custom circuit construction

bb = BbCode(
    l=15,
    m=3,
    A_x_pows=[9],
    A_y_pows=[1, 2],
    B_x_pows=[2, 7],
    B_y_pows=[0],
)
custom_circuit = bb.build_circuit(strategy="custom", num_rounds=1, basis="Z")
print("BB custom depth:", bb.depth)
print("Circuit instructions:", len(custom_circuit))


BB custom depth: 7
Circuit instructions: 122


In [5]:
# ErrorModel + CircuitBuildOptions example

error_model = ErrorModel(
    idle_error=1e-3,
    sqgate_error=1e-3,
    tqgate_error=1e-3,
    spam_error=1e-3,
)
build_opts = CircuitBuildOptions(
    get_all_detectors=False,
    noisy_zeroth_round=True,
    noisy_final_meas=False,
)

noisy_circuit = hgp.build_circuit(
    strategy="zxcoloration",
    error_model=error_model,
    num_rounds=2,
    basis="Z",
    circuit_build_options=build_opts,
)
print("Noisy circuit instructions:", len(noisy_circuit))


Noisy circuit instructions: 76


In [6]:
# Minimal decoder invocation (tiny run)

num_rounds = 3
num_trials = 100

circuit = hgp.build_circuit(
    strategy="zxcoloration",
    error_model=ErrorModel(
        idle_error=1e-3, 
        sqgate_error=1e-3,
        tqgate_error=1e-3,
        spam_error=1e-3,
    ),
    num_rounds=num_rounds,
    basis="Z",
)
detection_events, observable_flips = get_stim_mem_result(
    circuit, 
    num_trials=num_trials, 
    seed=1)

logical_pred = sliding_window_bposd_circuit_mem(
    detection_events,
    circuit,
    hgp.hz,
    hgp.lz,
    W=3,
    F=2,
    max_iter=10,
    osd_order=1,
    tqdm_on=False,
)

pL = np.mean((observable_flips - logical_pred).any(axis=1))
print({"num_trials": num_trials, "pL": float(pL)})


{'num_trials': 100, 'pL': 0.0}


## Where To Go Next

- Code family basics: [01A_codes_basics.ipynb](./01A_codes_basics.ipynb)
- Build your own code: [01B_make_my_own_code.ipynb](./01B_make_my_own_code.ipynb)
- Custom strategy details: [02A_custom_circuit_generation.ipynb](./02A_custom_circuit_generation.ipynb)
- Cardinal strategy details: [02B_cardinal_circuit_generation.ipynb](./02B_cardinal_circuit_generation.ipynb)
- ZX-coloration strategy details: [02C_zxcoloration_circuit_generation.ipynb](./02C_zxcoloration_circuit_generation.ipynb)
- Simulation pipeline: [03_simulation_pipeline.ipynb](./03_simulation_pipeline.ipynb)
- Decoder walkthroughs: [04_decoding_sliding_window.ipynb](./04_decoding_sliding_window.ipynb), [05_decoder_variants.ipynb](./05_decoder_variants.ipynb)

- End-to-end demos: [06A_end_to_end_demo_hgp.ipynb](./06A_end_to_end_demo_hgp.ipynb), [06B_end_to_end_demo_bb.ipynb](./06B_end_to_end_demo_bb.ipynb)
